# Cleaning and Storing Data in SQLite

In [58]:
#allows me to use django models
import sys
import os
import django
sys.path.append(os.path.dirname(os.path.abspath('')))
os.environ.setdefault('DJANGO_SETTINGS_MODULE','calgary_demo.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

#import the models
from olympic.models import Teams, Players, Games, Shots, Passes, Takeaways, Recoveries, \
    Penalties, Faceoffs, EE, PlayerGame, PlayerTourney, TeamGame, TeamTourney
    
#generic imports
import pandas as pd
import numpy as np
from scipy import stats
import requests
from django.db.models import Count, Max, Avg, Sum, Q, F, When, Case
import urllib.request

from django.shortcuts import render
from django.http import HttpResponse
from django.templatetags.static import static
from olympic.models import PlayerTourney, Teams, TeamTourney, Shots, Passes, Takeaways, Recoveries, Penalties, Faceoffs, EE, Players, Games
from django.db.models import Q
import numpy as np
import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Select, Slider, Select, MultiSelect, MultiChoice
from bokeh.plotting import figure, show, output_file, save
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.core.properties import value

### Read in the Data

In [36]:
#read in the data into dataframes and filter out NCAA games
df = pd.read_csv("olympic_womens_dataset.csv")
df = df[~df['Home Team'].isin(['Clarkson Golden Knights', 'St. Lawrence Saints'])]
df = df[~df['game_date'].str.startswith('2019')]

#simple name cleaning
df['Home Team'] = df['Home Team'].apply(lambda x: x.replace("Olympic (Women) - ", ""))
df['Away Team'] = df['Away Team'].apply(lambda x: x.replace("Olympic (Women) - ", ""))
df['Team'] = df['Team'].apply(lambda x: x.replace("Olympic (Women) - ", ""))

#adding in game and period time
def get_seconds(time_str, period):
    seconds = (3-period) * 1200
    m,s = time_str.split(":")
    seconds += int(m)*60 + int(s)
    return seconds

df['periodtime'] = df['Clock'].apply(lambda x: get_seconds(x,3))
df['gametime'] = df.apply(lambda row: get_seconds(row['Clock'],row['Period']), axis=1)
df['zone'] = df['X Coordinate'].apply(lambda x: 'dz' if x < 71 else ('nz' if x <= 129 else 'oz'))
df['zone_2'] = df['X Coordinate 2'].apply(lambda x: 'dz' if x < 71 else ('nz' if x <= 129 else 'oz'))
df['possession_count'] = ((df.Team != df.Team.shift()).cumsum())
df

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2,periodtime,gametime,zone,zone_2,possession_count
0,2018-02-11,Canada,Olympic Athletes from Russia,1,20:00,5,5,0,0,Canada,...,NaN,NaN,Lyudmila Belyakova,NaN,NaN,1200,3600,nz,oz,1
1,2018-02-11,Canada,Olympic Athletes from Russia,1,19:59,5,5,0,0,Canada,...,NaN,NaN,NaN,NaN,NaN,1199,3599,nz,oz,1
2,2018-02-11,Canada,Olympic Athletes from Russia,1,19:59,5,5,0,0,Canada,...,NaN,NaN,NaN,NaN,NaN,1199,3599,nz,oz,1
3,2018-02-11,Canada,Olympic Athletes from Russia,1,19:59,5,5,0,0,Canada,...,NaN,NaN,Liana Ganeyeva,NaN,NaN,1199,3599,nz,oz,1
4,2018-02-11,Canada,Olympic Athletes from Russia,1,19:53,5,5,0,0,Canada,...,NaN,NaN,NaN,NaN,NaN,1193,3593,oz,oz,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,2018-02-21,United States,Canada,4,0:09,3,4,2,2,United States,...,NaN,NaN,NaN,NaN,NaN,9,-1191,dz,oz,2112
9340,2018-02-21,United States,Canada,4,0:09,3,4,2,2,United States,...,NaN,NaN,Brianna Decker,26.0,82.0,9,-1191,dz,dz,2112
9341,2018-02-21,United States,Canada,4,0:08,3,4,2,2,United States,...,NaN,NaN,NaN,NaN,NaN,8,-1192,dz,oz,2112
9342,2018-02-21,United States,Canada,4,0:04,3,4,2,2,Canada,...,NaN,NaN,NaN,NaN,NaN,4,-1196,dz,oz,2113


### Create Teams Model

In [37]:
#get the unique team names and remove prefix
team_names = sorted(df['Team'].unique().tolist())
print(team_names)

['Canada', 'Finland', 'Olympic Athletes from Russia', 'United States']


In [38]:
#create simple data structure to build off of (hardcoded bc of flag_url)
teams_data_list = [{'team_id':0, 'name':team_names[0], 'flag_url':'images/canada.png'},
         {'team_id':1, 'name':team_names[1], 'flag_url':'images/finland.png'},
         {'team_id':2, 'name':team_names[2], 'flag_url':'images/roc.jpeg'},
         {'team_id':3, 'name':team_names[3], 'flag_url':'images/usa.png'}]

#now load into Django
teams_list = [Teams(**t) for t in teams_data_list]
#teams = Teams.objects.bulk_create(teams_list)

### Create Players Model

In [39]:
#get team objects WITH POSITIONS
all_teams_obj = list(Teams.objects.all())
team_name_mapper = {
    'Canada':all_teams_obj[0],
    'Finland':all_teams_obj[1],
    'Olympic Athletes from Russia':all_teams_obj[2],
    'United States':all_teams_obj[3],
}

#fill in team objects as foreign keys
players_df = df.loc[:,['Player', 'Team']].drop_duplicates(subset=['Player', 'Team'], keep='first')
players_df['Team'] = players_df['Team'].apply(lambda x: team_name_mapper[x])
players_df = players_df.reset_index().drop(['index'], axis=1)
players_df = players_df.rename(columns={'Player':'name',
                                        'Team':'team_id'})
players_df['player_id'] = players_df.index
players_data_list = players_df.to_dict('records')
players_list = [Players(**p) for p in players_data_list]
#players = Players.objects.bulk_create(players_list)

### Create Games Model

In [40]:
#simple aggregation to get games and final scores
games_df = df.groupby(['game_date', 'Home Team', 'Away Team']).agg({'Home Team Goals':'max', 'Away Team Goals':'max'})

#data cleaning
games_df = games_df.reset_index()
games_df['game_id'] = games_df.index
games_df = games_df.rename(columns={'game_date':'date',
                                    'Home Team': 'home_team', 'Away Team':'away_team',
                                    'Home Team Goals':'home_score', 'Away Team Goals':'away_score'})
games_df['home_team'] = games_df['home_team'].apply(lambda x: team_name_mapper[x])
games_df['away_team'] = games_df['away_team'].apply(lambda x: team_name_mapper[x])
games_data_list = games_df.to_dict('records')
games_list = [Games(**g) for g in games_data_list]
#games = Games.objects.bulk_create(games_list)

In [41]:
#add team ids to df
all_games_obj = Games.objects.all()
game_id_mapper = {
    '2018-02-11':0,
    '2018-02-13':1,
    '2018-02-14':2,
    '2018-02-19':3,
    '2018-02-21':4
}
df['game_id'] = df['game_date'].apply(lambda x: game_id_mapper[x])

### Create Passes Model

In [ ]:
#filter relevant events
passes_df = df[df.Event.isin(['Play', 'Incomplete Play'])]

#clean the data 
passes_df['success'] = passes_df['Event'].apply(lambda x: True if x == 'Play' else False)
passes_df = passes_df.drop(['game_date', 'Clock', 'Detail 2', 'Detail 3', 'Detail 4',
                            'Home Team', 'Away Team', 'Event', 'Home Team Goals',
                            'Away Team Goals', 'possession_count'], axis=1)
passes_df = passes_df.reset_index().drop(['index'], axis=1)
passes_df['pass_id'] = passes_df.index
passes_df = passes_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'passer_id', 'X Coordinate':'passer_x_coord',
                                      'Y Coordinate':'passer_y_coord','Detail 1':'pass_type',
                                      'Player 2':'receiver_id', 'X Coordinate 2':'receiver_x_coord',
                                      'Y Coordinate 2':'receiver_y_coord', 'zone':'passer_zone',
                                      'zone_2':'receiver_zone'})

#handle foreign keys
passes_df['team_id'] = passes_df['team_id'].apply(lambda x: Teams.objects.filter(name = x)[0])
passes_df['game_id'] = passes_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])
passes_df['passer_id'] = passes_df['passer_id'].apply(lambda x: Players.objects.filter(name = x)[0])
passes_df['receiver_id'] = passes_df['receiver_id'].apply(lambda x: Players.objects.filter(name = x)[0])
passes_data_list = passes_df.to_dict('records')
passes_list = [Passes(**p) for p in passes_data_list]
#passes = Passes.objects.bulk_create(passes_list)

### Create Takeaways Model

In [9]:
#filter relevant events
takeaways_df = df[df.Event.isin(['Takeaway'])]

#clean the data 
takeaways_df = takeaways_df.drop(['game_date', 'Clock', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4',
                            'Home Team', 'Away Team', 'Event', 'Home Team Goals', 'zone_2', 'possession_count',
                            'Away Team Goals', 'X Coordinate 2', 'Y Coordinate 2', 'Player 2'], axis=1)
takeaways_df = takeaways_df.reset_index().drop(['index'], axis=1)
takeaways_df['takeaway_id'] = takeaways_df.index
takeaways_df = takeaways_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'player_id', 'X Coordinate':'x_coord',
                                      'Y Coordinate':'y_coord', 'zone':'zone'})
#handle foreign keys
takeaways_df['team_id'] = takeaways_df['team_id'].apply(lambda x: Teams.objects.filter(name = x)[0])
takeaways_df['game_id'] = takeaways_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])
takeaways_df['player_id'] = takeaways_df['player_id'].apply(lambda x: Players.objects.filter(name = x)[0])
takeaways_data_list = takeaways_df.to_dict('records')
takeaways_list = [Takeaways(**t) for t in takeaways_data_list]
#takeaways = Takeaways.objects.bulk_create(takeaways_list)

### Create Recoveries Model

In [10]:
#filter relevant events
recoveries_df = df[df.Event.isin(['Puck Recovery'])]

#clean the data 
recoveries_df = recoveries_df.drop(['game_date', 'Clock', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4',
                            'Home Team', 'Away Team', 'Event', 'Home Team Goals', 'zone_2', 'possession_count',
                            'Away Team Goals', 'X Coordinate 2', 'Y Coordinate 2', 'Player 2'], axis=1)
recoveries_df = recoveries_df.reset_index().drop(['index'], axis=1)
recoveries_df['recovery_id'] = recoveries_df.index
recoveries_df = recoveries_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'player_id', 'X Coordinate':'x_coord',
                                      'Y Coordinate':'y_coord', 'zone':'zone'})
#handle foreign keys
recoveries_df['team_id'] = recoveries_df['team_id'].apply(lambda x: Teams.objects.filter(name = x)[0])
recoveries_df['game_id'] = recoveries_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])
recoveries_df['player_id'] = recoveries_df['player_id'].apply(lambda x: Players.objects.filter(name = x)[0])
recoveries_df
recoveries_data_list = recoveries_df.to_dict('records')
recoveries_list = [Recoveries(**r) for r in recoveries_data_list]
#recoveries = Recoveries.objects.bulk_create(recoveries_list)

### Create Penalties Model

In [ ]:
#filter relevant events
penalties_df = df[df.Event.isin(['Penalty Taken'])]

#create two rows per penalty (drawn vs taken)
penalties_df = pd.DataFrame(np.repeat(penalties_df.values,2, axis=0), columns=penalties_df.columns)
def apply_odd(row, field_odd, field_even):
    if row.name % 2 == 1: return field_odd
    else: return field_even
    
def change_zone(row):
    if row.name % 2 == 1:
        if row['zone'] == 'oz': return'dz'
        elif row['zone'] == 'dz': return'oz'
        else: return 'nz'
    else:
        return row['zone']

#update the player fields and the team fields for new rows
penalties_df['Player'] = penalties_df.apply(lambda row: apply_odd(row, row['Player 2'], row['Player']), axis=1)
penalties_df['other_team'] = penalties_df.apply(lambda row: row['Home Team'] if row['Home Team'] != row['Team'] else row['Away Team'], axis=1)
penalties_df['Team'] = penalties_df.apply(lambda row: apply_odd(row, row['other_team'], row['Team']), axis=1)
penalties_df['Zone'] = penalties_df.apply(lambda row: change_zone(row), axis=1)
penalties_df['X Coordinate'] = penalties_df.apply(lambda row: apply_odd(row, 200-row['X Coordinate'],row['X Coordinate']), axis=1)
penalties_df['Y Coordinate'] = penalties_df.apply(lambda row: apply_odd(row, 85-row['Y Coordinate'], row['Y Coordinate']), axis=1)
penalties_df['taken'] = penalties_df.apply(lambda row: apply_odd(row, False, True), axis=1)



#clean the data 
penalties_df = penalties_df.drop(['game_date', 'Clock', 'Detail 2', 'Detail 3', 'Detail 4', 'Zone',
                            'Home Team', 'Away Team', 'Event', 'Home Team Goals', 'zone_2', 'possession_count',
                            'Away Team Goals', 'X Coordinate 2', 'Y Coordinate 2', 'Player 2', 'other_team'], axis=1)
#penalties_df = penalties_df.reset_index().drop(['index'], axis=1)
penalties_df = penalties_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'player_id', 'X Coordinate':'x_coord',
                                      'Y Coordinate':'y_coord', 'zone':'zone', 'Detail 1': 'infraction_type'})
penalties_df = penalties_df.reset_index().drop(['index'], axis=1)
penalties_df['penalty_id'] = (penalties_df.index / 2).astype(int) 
penalties_df 

#handle foreign keys
penalties_df['team_id'] = penalties_df['team_id'].apply(lambda x: Teams.objects.filter(name = x)[0])
penalties_df['game_id'] = penalties_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])

#handle blameless penalties
def blameless(x):
    try:
        return Players.objects.filter(name = x)[0]
    except:
        return
    
    
penalties_df['player_id'] = penalties_df['player_id'].apply(lambda x: blameless(x))
penalties_data_list = penalties_df.to_dict('records')
penalties_list = [Penalties(**p) for p in penalties_data_list]
#penalties = Penalties.objects.bulk_create(penalties_list)

In [42]:
def apply_odd(row, field_odd, field_even):
    if row.name % 2 == 1: return field_odd
    else: return field_even
    
def change_zone(row):
    if row.name % 2 == 1:
        if row['zone'] == 'oz': return'dz'
        elif row['zone'] == 'dz': return'oz'
        else: return 'nz'
    else:
        return row['zone']
    
def blameless(x):
    try:
        return Players.objects.filter(name = x)[0]
    except:
        return

### Create Faceoffs Model

In [12]:
#filter relevant events
faceoffs_df = df[df.Event.isin(['Faceoff Win'])]

#create two rows per penalty (drawn vs taken)
faceoffs_df = pd.DataFrame(np.repeat(faceoffs_df.values,2, axis=0), columns=faceoffs_df.columns)

#update the player fields and the team fields for new rows
faceoffs_df['Player'] = faceoffs_df.apply(lambda row: apply_odd(row, row['Player 2'], row['Player']), axis=1)
faceoffs_df['other_team'] = faceoffs_df.apply(lambda row: row['Home Team'] if row['Home Team'] != row['Team'] else row['Away Team'], axis=1)
faceoffs_df['Team'] = faceoffs_df.apply(lambda row: apply_odd(row, row['other_team'], row['Team']), axis=1)
faceoffs_df['zone'] = faceoffs_df.apply(lambda row: change_zone(row), axis=1)
faceoffs_df['X Coordinate'] = faceoffs_df.apply(lambda row: apply_odd(row, 200-row['X Coordinate'],row['X Coordinate']), axis=1)
faceoffs_df['Y Coordinate'] = faceoffs_df.apply(lambda row: apply_odd(row, 85-row['Y Coordinate'], row['Y Coordinate']), axis=1)
faceoffs_df['success'] = faceoffs_df.apply(lambda row: apply_odd(row, False, True), axis=1)

#clean the data 
faceoffs_df = faceoffs_df.drop(['game_date', 'Clock', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4', 
                            'Home Team', 'Away Team', 'Event', 'Home Team Goals', 'zone_2', 'possession_count',
                            'Away Team Goals', 'X Coordinate 2', 'Y Coordinate 2', 'Player 2', 'other_team'], axis=1)
faceoffs_df = faceoffs_df.reset_index().drop(['index'], axis=1)
faceoffs_df = faceoffs_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'player_id', 'X Coordinate':'x_coord',
                                      'Y Coordinate':'y_coord'})

faceoffs_df = faceoffs_df.reset_index().drop(['index'], axis=1)
faceoffs_df['faceoff_id'] = (faceoffs_df.index / 2).astype(int) 

#handle foreign keys
faceoffs_df['team_id'] = faceoffs_df['team_id'].apply(lambda x: Teams.objects.filter(name = x)[0])
faceoffs_df['game_id'] = faceoffs_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])
faceoffs_df['player_id'] = faceoffs_df['player_id'].apply(lambda x: Players.objects.filter(name = x)[0])
faceoffs_data_list = faceoffs_df.to_dict('records')
faceoffs_list = [Faceoffs(**f) for f in faceoffs_data_list]
#faceoffs = Faceoffs.objects.bulk_create(faceoffs_list)

### Create Shots Model - Feature Engineer Assists

In [14]:
#filter relevant events
shots_df = df[df.Event.isin(['Shot', 'Goal'])]

#clean the data 
shots_df['result'] = shots_df.apply(lambda row: row['Detail 2'] if row['Event'] == 'Shot' else 'Goal', axis=1)
shots_df = shots_df.drop(['game_date', 'Clock', 'Home Team', 'Away Team', 'Event', 'Home Team Goals', 'zone_2', 'zone',
                            'Away Team Goals', 'X Coordinate 2', 'Y Coordinate 2', 'Player 2', 'Detail 2'], axis=1)
shots_df = shots_df.reset_index().drop(['index'], axis=1)
shots_df['shot_id'] = shots_df.index
shots_df = shots_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'player_id', 'X Coordinate':'x_coord',
                                      'Y Coordinate':'y_coord','Detail 1':'shot_type',
                                      'Detail 3':'traffic', 'Detail 4':'one_timer'})
shots_df['traffic'] = shots_df['traffic'].apply(lambda x: True if x == 't' else False)
shots_df['one_timer'] = shots_df['one_timer'].apply(lambda x: True if x == 't' else False)

#adding assists
def add_assists(row, assist_num):
    same_possession = df.loc[(df.possession_count == row['possession_count']),'Player'].values.tolist()[:-2]
    seen = set()
    assist_list = []
    for player in same_possession[::-1]:
        if (player not in seen) and (player != row['player_id']):
            assist_list.append(player)
            seen.add(player)
    if len(assist_list) >= assist_num:
        return assist_list[assist_num-1]
    else:
        return
    
shots_df['a1'] = shots_df.apply(lambda row: add_assists(row,1), axis=1)
shots_df['a2'] = shots_df.apply(lambda row: add_assists(row,2), axis=1)
shots_df['a3'] = shots_df.apply(lambda row: add_assists(row,3), axis=1)
shots_df = shots_df.drop(['possession_count'], axis=1)
 
#handle foreign keys
shots_df['team_id'] = shots_df['team_id'].apply(lambda x: Teams.objects.filter(name = x)[0])
shots_df['game_id'] = shots_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])
shots_df['player_id'] = shots_df['player_id'].apply(lambda x: Players.objects.filter(name = x)[0])
shots_df['a1'] = shots_df['a1'].apply(lambda x: blameless(x))
shots_df['a2'] = shots_df['a2'].apply(lambda x: blameless(x))
shots_df['a3'] = shots_df['a3'].apply(lambda x: blameless(x))
shots_data_list = shots_df.to_dict('records')
shots_list = [Shots(**s) for s in shots_data_list]
shots = Shots.objects.bulk_create(shots_list)

/var/folders/7g/32ny20b55tnfb3xrk1gr586h0000gn/T/ipykernel_38560/23871158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots_df['result'] = shots_df.apply(lambda row: row['Detail 2'] if row['Event'] == 'Shot' else 'Goal', axis=1)


### Create EE Model - Will Require Extra Engineering

In [43]:
EE.objects.all().delete()
PlayerTourney.objects.all().delete()
PlayerGame.objects.all().delete()
TeamGame.objects.all().delete()
TeamTourney.objects.all().delete()

(16, {'olympic.TeamTourney': 16})

In [44]:
#pass exits
px_df = df[(df.Event.isin(['Play'])) & (df.zone == 'dz') & (df.zone_2 != 'dz')]
fx_px_df = df[(df.Event.isin(['Incomplete Play'])) & (df.zone == 'dz') & (df.zone_2 != 'dz')]
px_df['type_ee'] = 'px'

#carried exits
cx_df = df.copy()
cx_df['prev_x_coord'] = cx_df['X Coordinate'].shift(1).fillna(300).astype(int)
cx_df['prev_possession_count'] = cx_df['possession_count'].shift(1).fillna(-1).astype(int)
cx_df = cx_df[~cx_df.Event.isin(['Play', 'Incomplete Play', 'Dump In/Out'])]
fe_cx_df = cx_df[cx_df.Event.isin(['Takeaway'])]
cx_df = cx_df[(cx_df['X Coordinate'] >= 71) & (cx_df['prev_x_coord'] < 71) & (cx_df['possession_count'] == cx_df['prev_possession_count'])]
fe_cx_df = fe_cx_df[((fe_cx_df['X Coordinate'] < 140) & (fe_cx_df['X Coordinate'] >= 129))]
cx_df = cx_df.drop(['prev_x_coord', 'prev_possession_count'], axis=1)
fx_cx_df = fe_cx_df.drop(['prev_x_coord', 'prev_possession_count'], axis=1)
cx_df['type_ee'] = 'cx'

#uncontrolled exits
ux_df = df.copy()
ux_df['next_event'] = ux_df['Event'].shift(-1).fillna('Zone Entry')
ux_df['next_x_coord'] = ux_df['X Coordinate'].shift(-1).fillna(300).astype(int)
ux_df = ux_df[ux_df.Event.isin(['Dump In/Out'])]
fe_ux_df = ux_df.copy()
ux_df = ux_df[(ux_df['next_event'] != 'Zone Entry') & (ux_df['zone'] == 'dz') & (ux_df['next_x_coord'] >= 71)]
fx_ux_df = fe_ux_df[(fe_ux_df['next_event'] != 'Zone Entry') & (fe_ux_df['zone'] == 'dz') & ((fe_ux_df['next_x_coord'] < 71) | (fe_ux_df['next_x_coord'] >= 129))]
ux_df = ux_df.drop(['next_event', 'next_x_coord'], axis=1)
fx_ux_df = fx_ux_df.drop(['next_event', 'next_x_coord'], axis=1)
ux_df['type_ee'] = 'ux'

#failed exits (negation of cx, px, ux)
fx_df = pd.concat([fx_cx_df, fx_ux_df, fx_px_df])
fx_df['type_ee'] = 'fx'

#controlled entries
ce_df = df.copy()
ce_df['next_x_coord'] = ce_df['X Coordinate'].shift(-1).fillna(300).astype(int)
ce_df['next_possession_count'] = ce_df['possession_count'].shift(-1).fillna(-1).astype(int)
fe_ce_df = ce_df.copy()
ce_df = ce_df[(ce_df.Event.isin(['Zone Entry'])) & (ce_df['Detail 1'].isin(['Carried','Played'])) &
              (ce_df['X Coordinate'] < 129) & (ce_df['next_x_coord'] >= 129) & (ce_df['possession_count'] == ce_df['next_possession_count'])]
fe_ce_df = fe_ce_df[(fe_ce_df.Event.isin(['Zone Entry'])) & (fe_ce_df['Detail 1'].isin(['Carried','Played'])) &
              (fe_ce_df['X Coordinate'] < 129) & ((fe_ce_df['next_x_coord'] < 129) | (fe_ce_df['possession_count'] != fe_ce_df['next_possession_count']))]
ce_df = ce_df.drop(['next_x_coord', 'next_possession_count'], axis=1)
fe_ce_df = fe_ce_df.drop(['next_x_coord', 'next_possession_count'], axis=1)
ce_df['type_ee'] = 'ce'

#uncontrolled entries
ue_df = df.copy()
ue_df['next_x_coord'] = ue_df['X Coordinate'].shift(-1).fillna(300).astype(int)
fe_ue_df = ue_df.copy()
ue_df = ue_df[(ue_df.Event.isin(['Zone Entry'])) & (ue_df['Detail 1'].isin(['Dumped'])) &
              (ue_df['X Coordinate'] < 129) & ((ue_df['next_x_coord'] >= 129) | (ue_df['next_x_coord'] < 71))]
fe_ue_df = fe_ue_df[(fe_ue_df.Event.isin(['Zone Entry'])) & (fe_ue_df['Detail 1'].isin(['Dumped'])) &
              (fe_ue_df['X Coordinate'] < 129) & ((fe_ue_df['next_x_coord'] < 129) & (fe_ue_df['next_x_coord'] >= 71))]
ue_df = ue_df.drop(['next_x_coord'], axis=1)
fe_ue_df = fe_ue_df.drop(['next_x_coord'], axis=1)
ue_df['type_ee'] = 'ue'

#failed entries (the negation of ce and ue)
fe_df = pd.concat([fe_ce_df, fe_ue_df])
fe_df['type_ee'] = 'fe'
print(cx_df.shape, ux_df.shape, fx_df.shape)

(362, 28) (116, 28) (347, 28)


/var/folders/7g/32ny20b55tnfb3xrk1gr586h0000gn/T/ipykernel_57152/3450023716.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  px_df['type_ee'] = 'px'


In [45]:
#now we combine all the ee and clean
ee_df = pd.concat([fe_df, ce_df, ue_df, cx_df, fx_df, px_df, ux_df])

#data cleaning
ee_df['defended_by_id'] = ee_df.apply(lambda row: blameless(row['Player 2']) if row['type_ee'] == 'ce' else None, axis=1)
ee_df = ee_df.drop(['game_date', 'Clock', 'Home Team', 'Away Team', 'Event', 'Home Team Goals', 'zone_2',
                            'Away Team Goals', 'X Coordinate 2', 'Y Coordinate 2', 'Player 2', 'Detail 2', 
                            'Detail 1', 'Detail 3', 'Detail 4', 'possession_count'], axis=1)
ee_df = ee_df.reset_index().drop(['index'], axis=1)
ee_df['ee_id'] = ee_df.index
ee_df = ee_df.rename(columns={'Period':'period', 'Home Team Skaters':'home_team_skaters',
                                      'Away Team Skaters':'away_team_skaters', 'Team':'team_id',
                                      'Player':'player_id', 'X Coordinate':'x_coord',
                                      'Y Coordinate':'y_coord'})

#handle foreign keys
def handle_name(x):
    try:
        return Teams.objects.filter(name = x)[0]
    except:
        return Teams.objects.filter(name = 'ROC')[0]
        
ee_df['team_id'] = ee_df['team_id'].apply(lambda x: handle_name(x))
ee_df['game_id'] = ee_df['game_id'].apply(lambda x: Games.objects.filter(game_id = x)[0])
ee_df['player_id'] = ee_df['player_id'].apply(lambda x: Players.objects.filter(name = x)[0])
ee_data_list = ee_df.to_dict('records')
ee_list = [EE(**ee) for ee in ee_data_list]
ee = EE.objects.bulk_create(ee_list)


## Aggregation Models

### Create PlayerGame Model

In [46]:
#get all games
all_games = Games.objects.all()

#loop through games and create
pg_list = []
index = 0
for g in all_games:
    
    #get the two teams in the game
    home_team = g.home_team
    away_team = g.away_team
    
    #get players from these teams
    players_on_teams = Players.objects.filter(Q(team_id = home_team) | Q(team_id = away_team))
    
    #loop through players
    for p in players_on_teams:
        
        #sanity check - don't create if they never passed the puck
        if len(Passes.objects.filter(Q(passer_id = p) & Q(game_id = g))) == 0:
            continue
        
        #loop through state variables
        for s in ['All', 'ES', 'PP', 'PK']:
        
            #properties
            pg_dict = {}
            pg_dict['pg_id'] = index
            index += 1
            pg_dict['game_id'] = g
            pg_dict['team_id'] = p.team_id
            pg_dict['player_id'] = p
            pg_dict['state'] = s
            base_cond = Q(player_id=p) & Q(game_id=g)
            pass_cond = Q(passer_id=p) & Q(game_id=g)
            
            #state condition
            if s == 'All':
                state_cond = Q(home_team_skaters__lt = 100)
            elif s == 'ES':
                state_cond = Q(home_team_skaters=F('away_team_skaters'))
            elif s == 'PP':
                if p.team_id == home_team:
                    state_cond = Q(home_team_skaters__gt=F('away_team_skaters'))
                else:
                    state_cond = Q(home_team_skaters__lt=F('away_team_skaters')) 
            else:
                if p.team_id == home_team:
                    state_cond = Q(home_team_skaters__lt=F('away_team_skaters'))
                else:
                    state_cond = Q(home_team_skaters__gt=F('away_team_skaters')) 
                    
            #base stats
            pg_dict['g'] = Shots.objects.filter(base_cond & state_cond & Q(result='Goal')).count()
            pg_dict['a'] = Shots.objects.filter((Q(a1=p) | Q(a2=p)) & state_cond & Q(result='Goal') & Q(game_id=g)).count()
            pg_dict['shots_for'] = Shots.objects.filter(base_cond & state_cond & (Q(result='Goal') | Q(result='On Net'))).count()
            try:
                pg_dict['shooting_percent'] = round(Shots.objects.filter(base_cond & state_cond & Q(result='Goal')).count() /\
                    Shots.objects.filter(base_cond & state_cond & (Q(result='Goal') | Q(result='On Net'))).count() ,2)
            except:
                pg_dict['shooting_percent'] = 0
            pg_dict['CF'] = Shots.objects.filter(base_cond & state_cond).count()
            pg_dict['total_passes'] = Passes.objects.filter(pass_cond & state_cond).count()
            pg_dict['successful_passes'] = Passes.objects.filter(pass_cond & state_cond & Q(success=True)).count()
            try:
                pg_dict['pass_completion'] = round(Passes.objects.filter(pass_cond & state_cond & Q(success=True)).count() /\
                    Passes.objects.filter(pass_cond & state_cond).count(),2)
            except:
                pg_dict['pass_completion'] = 0
            pg_dict['penalty_diff'] = Penalties.objects.filter(base_cond & state_cond & Q(taken=False)).count() - \
                Penalties.objects.filter(base_cond & state_cond & Q(taken=True)).count()
            pg_dict['oz_takeaway'] = Takeaways.objects.filter(base_cond & state_cond & Q(zone='oz')).count()
            pg_dict['dz_takeaway'] = Takeaways.objects.filter(base_cond & state_cond & Q(zone='dz')).count()
            pg_dict['oz_recovery'] = Recoveries.objects.filter(base_cond & state_cond & Q(zone='oz')).count()
            pg_dict['dz_recovery'] = Recoveries.objects.filter(base_cond & state_cond & Q(zone='dz')).count()
            
            #faceoff stats
            pg_dict['fo_taken'] = Faceoffs.objects.filter(base_cond & state_cond).count()
            pg_dict['fo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True)).count()
            if pg_dict['fo_taken'] > 0:
                pg_dict['fo_win_percent'] = round(pg_dict['fo_successful'] / pg_dict['fo_taken'], 2)
            else:
                pg_dict['fo_win_percent'] = 0
                
            pg_dict['nzfo_taken'] = Faceoffs.objects.filter(base_cond & state_cond & Q(zone='nz')).count()
            pg_dict['nzfo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True) & Q(zone='nz')).count()
            if pg_dict['nzfo_taken'] > 0:
                pg_dict['nzfo_win_percent'] = round(pg_dict['nzfo_successful'] / pg_dict['nzfo_taken'],2)
            else:
                pg_dict['nzfo_win_percent'] = 0
                
            pg_dict['ozfo_taken'] = Faceoffs.objects.filter(base_cond & state_cond & Q(zone='oz')).count()
            pg_dict['ozfo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True) & Q(zone='oz')).count()
            if pg_dict['ozfo_taken'] > 0:
                pg_dict['ozfo_win_percent'] = round(pg_dict['ozfo_successful'] / pg_dict['ozfo_taken'],2)
            else:
                pg_dict['ozfo_win_percent'] = 0
                
            pg_dict['dzfo_taken'] = Faceoffs.objects.filter(base_cond & state_cond & Q(zone='dz')).count()
            pg_dict['dzfo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True) & Q(zone='dz')).count()
            if pg_dict['dzfo_taken'] > 0:
                pg_dict['dzfo_win_percent'] = round(pg_dict['dzfo_successful'] / pg_dict['dzfo_taken'],2)
            else:
                pg_dict['dzfo_win_percent'] = 0
            
            #transition stats
            try:
                pg_dict['cx_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px', 'ux', 'fx'])).count() ,2)
            except:
                pg_dict['cx_rate'] = 0
            try:
                pg_dict['ux_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ux'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px', 'ux', 'fx'])).count() ,2)
            except:
                pg_dict['ux_rate'] = 0
            try:
                pg_dict['fx_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fx'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px', 'ux', 'fx'])).count() ,2)
            except:
                pg_dict['fx_rate'] = 0
            pg_dict['cx'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px'])).count()
            pg_dict['ux'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ux'])).count()
            pg_dict['fx'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fx'])).count()

            
            try:
                pg_dict['ce_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce', 'ue', 'fe'])).count() ,2)
            except:
                pg_dict['ce_rate'] = 0
            try:
                pg_dict['ue_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ue'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce', 'ue', 'fe'])).count() ,2)
            except:
                pg_dict['ue_rate'] = 0
            try:
                pg_dict['fe_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fe'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce', 'ue', 'fe'])).count() ,2)
            except:
                pg_dict['fe_rate'] = 0
            pg_dict['ce'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce'])).count()
            pg_dict['ue'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ue'])).count()
            pg_dict['fe'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fe'])).count()
            
            pg_list.append(pg_dict)
            
        
                        
#save to DB
pg_final = [PlayerGame(**pg) for pg in pg_list]
pg = PlayerGame.objects.bulk_create(pg_final)            

### Create PlayerTourney Model

In [47]:
#get players from these teams
all_players = Players.objects.filter(~Q(position='G'))

#loop through players
pt_list = []
index = 0
for p in all_players:
    
    #sanity check - don't create if they never passed the puck
    if len(Passes.objects.filter(Q(passer_id = p))) == 0:
        continue
    
    #loop through state variables
    for s in ['All', 'ES', 'PP', 'PK']:
    
        #properties
        pt_dict = {}
        pt_dict['pt_id'] = index
        index += 1
        pt_dict['team_id'] = p.team_id
        pt_dict['player_id'] = p
        pt_dict['state'] = s
        base_cond = Q(player_id=p) 
        pass_cond = Q(passer_id=p) 
        
        #state condition
        if s == 'All':
            state_cond = Q(state = 'All')
        elif s == 'ES':
            state_cond = Q(state = 'ES')
        elif s == 'PP':
            state_cond = Q(state = 'PP')
        else:
            state_cond = Q(state = 'PK')
            
        #base stats
        pt_dict['gp'] = PlayerGame.objects.filter(base_cond & state_cond).count()
        values = PlayerGame.objects.filter(base_cond & state_cond).aggregate(Avg('g'), Avg('a'), Avg('CF'), Avg('shots_for'), Sum('g'),
                                                                             Sum('shots_for'), Avg('total_passes'), Sum('total_passes'),
                                                                             Avg('successful_passes'), Sum('successful_passes'), Avg('penalty_diff'),
                                                                             Avg('oz_takeaway'), Avg('dz_takeaway'), Avg('oz_recovery'),
                                                                             Avg('dz_recovery'), Avg('fo_taken'), Avg('nzfo_taken'),
                                                                             Avg('dzfo_taken'), Avg('ozfo_taken'), Sum('fo_taken'),
                                                                             Sum('nzfo_taken'),Sum('dzfo_taken'),Sum('ozfo_taken'),
                                                                             Avg('fo_successful'), Avg('nzfo_successful'),
                                                                             Avg('dzfo_successful'), Avg('ozfo_successful'), Sum('fo_successful'),
                                                                             Sum('nzfo_successful'),Sum('dzfo_successful'),Sum('ozfo_successful'),
                                                                             Avg('cx'), Avg('fx'), Avg('ux'), Avg('ce'), Avg('ue'), Avg('fe'),
                                                                             Sum('cx'), Sum('fx'), Sum('ux'), Sum('ce'), Sum('ue'), Sum('fe'))
        pt_dict['g_gp'] = round(values['g__avg'],2)
        pt_dict['a_gp'] = round(values['a__avg'],2)
        pt_dict['CF_gp'] = round(values['CF__avg'],2)
        pt_dict['shots_for_gp'] = round(values['shots_for__avg'],2)
        try:
            pt_dict['shooting_percent'] = round(values['g__sum'] / values['shots_for__sum'],2)
        except:
            pt_dict['shooting_percent'] = 0
        pt_dict['total_passes_gp'] = round(values['total_passes__avg'],2)
        pt_dict['successful_passes_gp'] = round(values['successful_passes__avg'],2)
        try:
            pt_dict['pass_completion'] = round(values['successful_passes__sum'] / values['total_passes__sum'],2)
        except:
            pt_dict['pass_completion'] = 0
        pt_dict['penalty_diff_gp'] = round(values['penalty_diff__avg'],2)
        pt_dict['oz_takeaway_gp'] = round(values['oz_takeaway__avg'],2)
        pt_dict['dz_takeaway_gp'] = round(values['dz_takeaway__avg'],2)
        pt_dict['oz_recovery_gp'] = round(values['oz_recovery__avg'],2)
        pt_dict['dz_recovery_gp'] = round(values['dz_recovery__avg'],2)
        
        #faceoffs
        pt_dict['fo_taken_gp'] = round(values['fo_taken__avg'],2)
        pt_dict['nzfo_taken_gp'] = round(values['nzfo_taken__avg'],2)
        pt_dict['ozfo_taken_gp'] = round(values['ozfo_taken__avg'],2)
        pt_dict['dzfo_taken_gp'] = round(values['dzfo_taken__avg'],2)
        
        pt_dict['fo_successful_gp'] = round(values['fo_successful__avg'],2)
        pt_dict['nzfo_successful_gp'] = round(values['nzfo_successful__avg'],2)
        pt_dict['ozfo_successful_gp'] = round(values['ozfo_successful__avg'],2)
        pt_dict['dzfo_successful_gp'] = round(values['dzfo_successful__avg'],2)
        
        if values['fo_taken__sum'] > 0:
            pt_dict['fo_win_percent'] = round(values['fo_successful__sum'] / values['fo_taken__sum'],2)
        else:
            pt_dict['fo_win_percent'] = 0
            
        if values['nzfo_taken__sum'] > 0:
            pt_dict['nzfo_win_percent'] = round(values['nzfo_successful__sum'] / values['nzfo_taken__sum'],2)
        else:
            pt_dict['nzfo_win_percent'] = 0
            
        if values['ozfo_taken__sum'] > 0:
            pt_dict['ozfo_win_percent'] = round(values['ozfo_successful__sum'] / values['ozfo_taken__sum'],2)
        else:
            pt_dict['ozfo_win_percent'] = 0
            
        if values['dzfo_taken__sum'] > 0:
            pt_dict['dzfo_win_percent'] = round(values['dzfo_successful__sum'] / values['dzfo_taken__sum'],2)
        else:
            pt_dict['dzfo_win_percent'] = 0
        
        #transition stats
        pt_dict['cx_gp'] = round(values['cx__avg'],2)
        pt_dict['ux_gp'] = round(values['ux__avg'],2)
        pt_dict['fx_gp'] = round(values['fx__avg'],2)
        pt_dict['ce_gp'] = round(values['ce__avg'],2)
        pt_dict['ue_gp'] = round(values['ue__avg'],2)
        pt_dict['fe_gp'] = round(values['fe__avg'],2)
        
        if values['cx__sum'] + values['ux__sum'] + values['fx__sum'] > 0:
            pt_dict['cx_rate'] = round(values['cx__sum'] / (values['cx__sum'] + values['ux__sum'] + values['fx__sum']), 2)
            pt_dict['ux_rate'] = round(values['ux__sum'] / (values['cx__sum'] + values['ux__sum'] + values['fx__sum']), 2)
            pt_dict['fx_rate'] = round(values['fx__sum'] / (values['cx__sum'] + values['ux__sum'] + values['fx__sum']), 2)
        else:
            pt_dict['cx_rate'] = 0
            pt_dict['ux_rate'] = 0
            pt_dict['fx_rate'] = 0
            
        if values['ce__sum'] + values['ue__sum'] + values['fe__sum'] > 0:
            pt_dict['ce_rate'] = round(values['ce__sum'] / (values['ce__sum'] + values['ue__sum'] + values['fe__sum']), 2)
            pt_dict['ue_rate'] = round(values['ue__sum'] / (values['ce__sum'] + values['ue__sum'] + values['fe__sum']), 2)
            pt_dict['fe_rate'] = round(values['fe__sum'] / (values['ce__sum'] + values['ue__sum'] + values['fe__sum']), 2)
        else:
            pt_dict['ce_rate'] = 0
            pt_dict['ue_rate'] = 0   
            pt_dict['fe_rate'] = 0
        
        pt_list.append(pt_dict)
        
    
                    
#save to DB
pt_final = [PlayerTourney(**pt) for pt in pt_list]
pt = PlayerTourney.objects.bulk_create(pt_final)

In [48]:
#fills in the percentiles for each field
# requires: Model, List['string'] -> None
def percentiles_django(model, fields):
    
    #loop through states
    for s in ['All', 'ES', 'PP', 'PK']:
        
        #loop through fields
        for f in fields:
    
            #get relevant models
            sorted_objects = model.objects.filter(Q(state=s)).order_by(f)
            sorted_values = list(model.objects.filter(Q(state=s)).order_by(f).values_list(f, flat=True))
            
            #now loop through models and assign percentiles
            for obj in sorted_objects:
                setattr(obj, f+'_percentile', round(stats.percentileofscore(sorted_values, getattr(obj, f)))/100)
                obj.save()
                
percentiles_django(PlayerTourney, ['g_gp', 'a_gp', 'CF_gp', 'pass_completion', 'shooting_percent',
                                          'penalty_diff_gp', 'cx_rate', 'ce_rate', 'shots_for_gp'])

### Create TeamGame Model

In [49]:
#get all games
all_games = Games.objects.all()

#loop through games and create
tg_list = []
index = 0
for g in all_games:
    
    #get the two teams in the game
    home_team = g.home_team
    away_team = g.away_team
    
    #loop through players
    for t in [g.home_team, g.away_team]:
        
        #sanity check - don't create if they never passed the puck
        if len(Passes.objects.filter(Q(team_id = t) & Q(game_id = g))) == 0:
            continue
        
        #loop through state variables
        for s in ['All', 'ES', 'PP', 'PK']:
        
            #properties
            tg_dict = {}
            tg_dict['tg_id'] = index
            index += 1
            tg_dict['game_id'] = g
            tg_dict['team_id'] = t
            tg_dict['state'] = s
            base_cond = Q(team_id=t) & Q(game_id=g)
            opp_cond = ~Q(team_id=t) & Q(game_id=g)
            
            #state condition
            if s == 'All':
                state_cond = Q(home_team_skaters__lt = 100)
            elif s == 'ES':
                state_cond = Q(home_team_skaters=F('away_team_skaters'))
            elif s == 'PP':
                if t == g.home_team:
                    state_cond = Q(home_team_skaters__gt=F('away_team_skaters'))
                else:
                    state_cond = Q(home_team_skaters__lt=F('away_team_skaters')) 
            else:
                if t == g.home_team:
                    state_cond = Q(home_team_skaters__lt=F('away_team_skaters'))
                else:
                    state_cond = Q(home_team_skaters__gt=F('away_team_skaters')) 
                    
            #base stats
            tg_dict['gf'] = Shots.objects.filter(base_cond & state_cond & Q(result='Goal')).count()
            tg_dict['ga'] = Shots.objects.filter(opp_cond & state_cond & Q(result='Goal')).count()
            try:
                tg_dict['gf_percent'] = round(tg_dict['gf'] / (tg_dict['gf'] + tg_dict['ga']),2)
            except:
                tg_dict['gf_percent'] = 0
                
            tg_dict['CF'] = Shots.objects.filter(base_cond & state_cond).count()
            tg_dict['CA'] = Shots.objects.filter(opp_cond & state_cond).count()
            try:
                tg_dict['CF_percent'] = round(tg_dict['CF'] / (tg_dict['CF'] + tg_dict['CA']),2)
            except:
                tg_dict['CF_percent'] = 0
                
            tg_dict['shots_for'] = Shots.objects.filter(base_cond & state_cond & (Q(result='Goal') | Q(result='On Net'))).count()
            tg_dict['shots_against'] = Shots.objects.filter(opp_cond & state_cond & (Q(result='Goal') | Q(result='On Net'))).count()
            try:
                tg_dict['shots_share'] = round(tg_dict['shots_for'] / (tg_dict['shots_for'] + tg_dict['shots_against']),2)
            except:
                tg_dict['shots_share'] = 0
                
            try:
                tg_dict['shooting_percent'] = round(tg_dict['gf'] / tg_dict['shots_for'], 2)
            except:
                tg_dict['shooting_percent'] = 0
                
                
            tg_dict['FF'] = Shots.objects.filter(base_cond & state_cond & ~Q(result='Blocked')).count()
            tg_dict['FA'] = Shots.objects.filter(opp_cond & state_cond & ~Q(result='Blocked')).count()
            try:
                tg_dict['FF_percent'] = round(tg_dict['FF'] / (tg_dict['FF'] + tg_dict['FA']),2)
            except:
                tg_dict['FF_percent'] = 0
                
            tg_dict['total_passes'] = Passes.objects.filter(base_cond & state_cond).count()
            tg_dict['successful_passes'] = Passes.objects.filter(base_cond & state_cond & Q(success=True)).count()
            try:
                tg_dict['pass_completion'] = round(tg_dict['successful_passes'] / tg_dict['total_passes'],2)
            except:
                tg_dict['pass_completion'] = 0
                
            tg_dict['penalty_diff'] = Penalties.objects.filter(base_cond & state_cond & Q(taken=False)).count() - \
                Penalties.objects.filter(base_cond & state_cond & Q(taken=True)).count()
            tg_dict['puck_recoveries_won'] = Recoveries.objects.filter(base_cond & state_cond).count()
            tg_dict['puck_recoveries_lost'] = Recoveries.objects.filter(opp_cond & state_cond).count()
            tg_dict['screened_shots'] = Shots.objects.filter(base_cond & state_cond & Q(traffic=True) & (Q(result='Goal') | Q(result='On Net'))).count()
                
            #transition stats
            try:
                tg_dict['cx_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px', 'ux', 'fx'])).count() ,2)
            except:
                tg_dict['cx_rate'] = 0
            try:
                tg_dict['ux_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ux'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px', 'ux', 'fx'])).count() ,2)
            except:
                tg_dict['ux_rate'] = 0
            try:
                tg_dict['fx_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fx'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px', 'ux', 'fx'])).count() ,2)
            except:
                tg_dict['fx_rate'] = 0
            tg_dict['cx'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['cx','px'])).count()
            tg_dict['ux'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ux'])).count()
            tg_dict['fx'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fx'])).count()

            
            try:
                tg_dict['ce_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce', 'ue', 'fe'])).count() ,2)
            except:
                tg_dict['ce_rate'] = 0
            try:
                tg_dict['ue_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ue'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce', 'ue', 'fe'])).count() ,2)
            except:
                tg_dict['ue_rate'] = 0
            try:
                tg_dict['fe_rate'] = round(EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fe'])).count() /\
                    EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce', 'ue', 'fe'])).count() ,2)
            except:
                tg_dict['fe_rate'] = 0
            tg_dict['ce'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ce'])).count()
            tg_dict['ue'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['ue'])).count()
            tg_dict['fe'] = EE.objects.filter(base_cond & state_cond & Q(type_ee__in=['fe'])).count()
            tg_dict['ce_against'] = EE.objects.filter(opp_cond & state_cond & Q(type_ee__in=['ce'])).count()
            tg_dict['ue_against'] = EE.objects.filter(opp_cond & state_cond & Q(type_ee__in=['ue'])).count()
            tg_dict['fe_against'] = EE.objects.filter(opp_cond & state_cond & Q(type_ee__in=['fe'])).count()
            if tg_dict['fe_against'] + tg_dict['ue_against'] + tg_dict['ce_against'] > 0:
                tg_dict['ce_against_rate'] = round(tg_dict['ce_against'] / (tg_dict['fe_against'] + tg_dict['ue_against'] + tg_dict['ce_against']),2)
                tg_dict['zone_denial_rate'] = round(tg_dict['fe_against'] / (tg_dict['fe_against'] + tg_dict['ue_against'] + tg_dict['ce_against']),2)
            else:
                tg_dict['ce_against_rate'] = 0
                tg_dict['zone_denial_rate'] = 0
                
            #faceoff stats
            tg_dict['fo_taken'] = Faceoffs.objects.filter(base_cond & state_cond).count()
            tg_dict['fo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True)).count()
            if tg_dict['fo_taken'] > 0:
                tg_dict['fo_win_percent'] = round(tg_dict['fo_successful'] / tg_dict['fo_taken'], 2)
            else:
                tg_dict['fo_win_percent'] = 0
                
            tg_dict['nzfo_taken'] = Faceoffs.objects.filter(base_cond & state_cond & Q(zone='nz')).count()
            tg_dict['nzfo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True) & Q(zone='nz')).count()
            if tg_dict['nzfo_taken'] > 0:
                tg_dict['nzfo_win_percent'] = round(tg_dict['nzfo_successful'] / tg_dict['nzfo_taken'],2)
            else:
                tg_dict['nzfo_win_percent'] = 0
                
            tg_dict['ozfo_taken'] = Faceoffs.objects.filter(base_cond & state_cond & Q(zone='oz')).count()
            tg_dict['ozfo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True) & Q(zone='oz')).count()
            if tg_dict['ozfo_taken'] > 0:
                tg_dict['ozfo_win_percent'] = round(tg_dict['ozfo_successful'] / tg_dict['ozfo_taken'],2)
            else:
                tg_dict['ozfo_win_percent'] = 0
                
            tg_dict['dzfo_taken'] = Faceoffs.objects.filter(base_cond & state_cond & Q(zone='dz')).count()
            tg_dict['dzfo_successful'] = Faceoffs.objects.filter(base_cond & state_cond & Q(success=True) & Q(zone='dz')).count()
            if tg_dict['dzfo_taken'] > 0:
                tg_dict['dzfo_win_percent'] = round(tg_dict['dzfo_successful'] / tg_dict['dzfo_taken'],2)
            else:
                tg_dict['dzfo_win_percent'] = 0                
                
            
            tg_list.append(tg_dict)
                        
#save to DB
tg_final = [TeamGame(**tg) for tg in tg_list]
tg = TeamGame.objects.bulk_create(tg_final)            

### Create TeamTourney Model

In [50]:
#get players from these teams
all_teams = Teams.objects.all()

#loop through players
tt_list = []
index = 0
for t in all_teams:
    
    #sanity check - don't create if they never passed the puck
    if len(Passes.objects.filter(Q(team_id = t))) == 0:
        continue
    
    #loop through state variables
    for s in ['All', 'ES', 'PP', 'PK']:
    
        #properties
        tt_dict = {}
        tt_dict['tt_id'] = index
        index += 1
        tt_dict['team_id'] = t
        tt_dict['state'] = s
        base_cond = Q(team_id=t) 
        opp_cond = ~Q(team_id=t) & (Q(game_id__home_team=t) | Q(game_id__away_team=t))
        
        #state condition
        if s == 'All':
            state_cond = Q(state = 'All')
        elif s == 'ES':
            state_cond = Q(state = 'ES')
        elif s == 'PP':
            state_cond = Q(state = 'PP')
        else:
            state_cond = Q(state = 'PK')
            
        #base stats
        tt_dict['gp'] = TeamGame.objects.filter(base_cond & state_cond).count()
        values = TeamGame.objects.filter(base_cond & state_cond).aggregate(Avg('gf'), Avg('ga'), Avg('CF'), Avg('CA'), Sum('CF'), Sum('CA')  ,Avg('shots_for'), Avg('shots_against'),
                                                                             Sum('shots_for'), Sum('shots_against'),Avg('total_passes'), Sum('total_passes'), Sum('gf'), Sum('ga'),
                                                                            Avg('FF'), Avg('FA'), Sum('FF'), Sum('FA'),
                                                                            Avg('successful_passes'), Sum('successful_passes'), Avg('total_passes'), Sum('total_passes'),
                                                                             Avg('fo_taken'), Avg('nzfo_taken'),
                                                                             Avg('dzfo_taken'), Avg('ozfo_taken'), Sum('fo_taken'),
                                                                             Sum('nzfo_taken'),Sum('dzfo_taken'),Sum('ozfo_taken'),
                                                                             Avg('fo_successful'), Avg('nzfo_successful'),
                                                                             Avg('dzfo_successful'), Avg('ozfo_successful'), Sum('fo_successful'),
                                                                             Sum('nzfo_successful'),Sum('dzfo_successful'),Sum('ozfo_successful'),
                                                                             Avg('cx'), Avg('fx'), Avg('ux'), Avg('ce'), Avg('ue'), Avg('fe'),
                                                                             Sum('cx'), Sum('fx'), Sum('ux'), Sum('ce'), Sum('ue'), Sum('fe'),
                                                                             Avg('ce_against'), Sum('ce_against'), Avg('ue_against'), Sum('ue_against'), Avg('fe_against'), Sum('fe_against'))
        tt_dict['gf_gp'] = round(values['gf__avg'],2)
        tt_dict['ga_gp'] = round(values['ga__avg'],2)
        try:
            tt_dict['gf_percent'] = round((values['gf__sum'])/(values['gf__sum'] + values['ga__sum']),2)
        except:
            tt_dict['gf_percent'] = 0
        
        tt_dict['CF_gp'] = round(values['CF__avg'],2)
        tt_dict['CA_gp'] = round(values['CA__avg'],2)
        if (values['CF__sum'] + values['CA__sum']) > 0:
            tt_dict['CF_percent'] = round(values['CF__sum']/(values['CF__sum'] + values['CA__sum']),2)
        else:
            tt_dict['CF_percent'] = 0
            
        tt_dict['FF_gp'] = round(values['FF__avg'],2)
        tt_dict['FA_gp'] = round(values['FA__avg'],2)
        if (values['FF__sum'] + values['FA__sum']) > 0:
            tt_dict['FF_percent'] = round(values['FF__sum']/(values['FF__sum'] + values['FA__sum']),2)
        else:
            tt_dict['FF_percent'] = 0
            
        tt_dict['shots_for_gp'] = round(values['shots_for__avg'],2)
        tt_dict['shots_against_gp'] = round(values['shots_against__avg'],2)
        if (values['shots_for__sum'] + values['shots_against__sum']) > 0:
            tt_dict['shots_share_gp'] = round(values['shots_for__sum']/(values['shots_for__sum'] + values['shots_against__sum']),2)
        else:
            tt_dict['shots_share_gp'] = 0
            
        try:
            tt_dict['shooting_percent'] = round(values['gf__sum'] / values['shots_for__sum'],3)
        except:
           tt_dict['shooting_percent'] = 0
            
        try:
            tt_dict['save_percentage'] = round((values['shots_against__sum'] - values['ga__sum']) / values['shots_against__sum'],3)
        except:
            tt_dict['save_percentage'] = 0
            
        try:
            tt_dict['PDO'] = round(((values['shots_against__sum'] - values['ga__sum']) / values['shots_against__sum']) + (values['gf__sum'] / values['shots_for__sum']),3)
        except:
            tt_dict['PDO'] = 0
        
        #transition stats
        tt_dict['cx_gp'] = round(values['cx__avg'],2)
        tt_dict['ux_gp'] = round(values['ux__avg'],2)
        tt_dict['fx_gp'] = round(values['fx__avg'],2)
        tt_dict['ce_gp'] = round(values['ce__avg'],2)
        tt_dict['ue_gp'] = round(values['ue__avg'],2)
        tt_dict['fe_gp'] = round(values['fe__avg'],2)
        tt_dict['ce_against_gp'] = round(values['ce_against__avg'],2)
        tt_dict['ue_against_gp'] = round(values['ue_against__avg'],2)
        tt_dict['fe_against_gp'] = round(values['fe_against__avg'],2)
        
        if values['cx__sum'] + values['ux__sum'] + values['fx__sum'] > 0:
            tt_dict['cx_rate'] = round(values['cx__sum'] / (values['cx__sum'] + values['ux__sum'] + values['fx__sum']), 2)
            tt_dict['ux_rate'] = round(values['ux__sum'] / (values['cx__sum'] + values['ux__sum'] + values['fx__sum']), 2)
            tt_dict['fx_rate'] = round(values['fx__sum'] / (values['cx__sum'] + values['ux__sum'] + values['fx__sum']), 2)
        else:
            tt_dict['cx_rate'] = 0
            tt_dict['ux_rate'] = 0
            tt_dict['fx_rate'] = 0
            
        if values['ce__sum'] + values['ue__sum'] + values['fe__sum'] > 0:
            tt_dict['ce_rate'] = round(values['ce__sum'] / (values['ce__sum'] + values['ue__sum'] + values['fe__sum']), 2)
            tt_dict['ue_rate'] = round(values['ue__sum'] / (values['ce__sum'] + values['ue__sum'] + values['fe__sum']), 2)
            tt_dict['fe_rate'] = round(values['fe__sum'] / (values['ce__sum'] + values['ue__sum'] + values['fe__sum']), 2)
        else:
            tt_dict['ce_rate'] = 0
            tt_dict['ue_rate'] = 0   
            tt_dict['fe_rate'] = 0
            
        if values['ce_against__sum'] + values['ue_against__sum'] + values['fe_against__sum'] > 0:
            tt_dict['ce_against_rate'] = round(values['ce_against__sum'] / (values['ce_against__sum'] + values['ue_against__sum'] + values['fe_against__sum']), 2)
            tt_dict['zone_denial_rate'] = round(values['fe_against__sum'] / (values['ce_against__sum'] + values['ue_against__sum'] + values['fe_against__sum']), 2)
        else:
            tt_dict['ce_against_rate'] = 0
            tt_dict['zone_denial_rate'] = 0   
            
        #faceoffs
        tt_dict['fo_taken_gp'] = round(values['fo_taken__avg'],2)
        tt_dict['nzfo_taken_gp'] = round(values['nzfo_taken__avg'],2)
        tt_dict['ozfo_taken_gp'] = round(values['ozfo_taken__avg'],2)
        tt_dict['dzfo_taken_gp'] = round(values['dzfo_taken__avg'],2)
        
        tt_dict['fo_successful_gp'] = round(values['fo_successful__avg'],2)
        tt_dict['nzfo_successful_gp'] = round(values['nzfo_successful__avg'],2)
        tt_dict['ozfo_successful_gp'] = round(values['ozfo_successful__avg'],2)
        tt_dict['dzfo_successful_gp'] = round(values['dzfo_successful__avg'],2)
        
        if values['fo_taken__sum'] > 0:
            tt_dict['fo_win_percent'] = round(values['fo_successful__sum'] / values['fo_taken__sum'],2)
        else:
            tt_dict['fo_win_percent'] = 0
            
        if values['nzfo_taken__sum'] > 0:
            tt_dict['nzfo_win_percent'] = round(values['nzfo_successful__sum'] / values['nzfo_taken__sum'],2)
        else:
            tt_dict['nzfo_win_percent'] = 0
            
        if values['ozfo_taken__sum'] > 0:
            tt_dict['ozfo_win_percent'] = round(values['ozfo_successful__sum'] / values['ozfo_taken__sum'],2)
        else:
            tt_dict['ozfo_win_percent'] = 0
            
        if values['dzfo_taken__sum'] > 0:
            tt_dict['dzfo_win_percent'] = round(values['dzfo_successful__sum'] / values['dzfo_taken__sum'],2)
        else:
            tt_dict['dzfo_win_percent'] = 0
            
        #passing
        tt_dict['total_passes_gp'] = round(values['total_passes__avg'],2)
        tt_dict['successful_passes_gp'] = round(values['successful_passes__avg'],2)
        try:
            tt_dict['pass_completion'] = round(values['successful_passes__sum'] / values['total_passes__sum'],2)
        except:
            tt_dict['pass_completion'] = 0
                    
        tt_list.append(tt_dict) 
    
                    
#save to DB
tt_final = [TeamTourney(**tt) for tt in tt_list]
tt = TeamTourney.objects.bulk_create(tt_final)

In [51]:
def ranks_django(model, fields, truth_list):
    
    #loop through states
    for s in ['All', 'ES', 'PP', 'PK']:
        
        #loop through fields
        for f, z in zip(fields, truth_list):
    
            #get relevant models
            if z == True:
                sorted_objects = model.objects.filter(Q(state=s)).order_by('-'+f)
            else:
                sorted_objects = model.objects.filter(Q(state=s)).order_by(f)
            
            #now loop through models and assign percentiles
            for rank, obj in enumerate(sorted_objects,1):
                setattr(obj, f+'_percentile', rank)
                obj.save()

ranks_django(TeamTourney, ['gf_gp', 'ga_gp', 'CF_gp', 'CA_gp', 'CF_percent', 'pass_completion', 'shooting_percent',
                                          'save_percentage', 'PDO', 'FF_gp', 'FA_gp', 'FF_percent', 'shots_against_gp',
                                          'shots_share_gp', 'cx_rate', 'ux_rate', 'fx_rate' ,'cx_gp', 'ux_gp', 'fx_gp',
                                          'ce_rate', 'ue_rate', 'fe_rate' ,'ce_gp', 'ue_gp', 'fe_gp', 'ce_against_gp',
                                          'ue_against_gp', 'fe_against_gp', 'ce_against_rate', 'zone_denial_rate',
                                          'fo_win_percent', 'nzfo_win_percent', 'ozfo_win_percent', 'dzfo_win_percent',
                                          'fo_taken_gp', 'nzfo_taken_gp', 'dzfo_taken_gp', 'ozfo_taken_gp', 'shots_for_gp',
                                          'fo_successful_gp', 'nzfo_successful_gp', 'dzfo_successful_gp', 'ozfo_successful_gp',
                                          'pass_completion', 'total_passes_gp', 'successful_passes_gp', 'gf_percent'],
                           [True, False, True, False, True, True, True, True, True, True, False, True, False,
                                          True, True, True, False ,True, True, False,
                                          True, True, False ,True, True, False, False,
                                          False, True, False, True,
                                          True, True, True, True,
                                          True, True, True, True, True,
                                          True, True, True, True,
                                          True, True, True, True])

In [59]:
context_dict = {}
    
#query all shots
event_list_dict = []
all_shots = Shots.objects.all()
for s in all_shots:
    
    #shots for
    d = {}
    d['Event'] = 'Shots'
    d['Perspective'] = 'For'
    d['Team'] = s.team_id.name
    d['Game'] = str(s.game_id.game_id)
    d['Player'] = s.player_id.name
    if s.home_team_skaters == s.away_team_skaters:
        d['State'] = 'ES'
    elif (s.home_team_skaters > s.away_team_skaters):
        if (s.team_id.name == s.game_id.home_team.name):
            d['State'] = 'PP'
        else:
            d['State'] = 'PK'
    else:
        if (s.team_id.name == s.game_id.away_team.name):
            d['State'] = 'PK'
        else:
            d['State'] = 'PP'
    if s.result == 'Goal':
        d['color'] = 'green'
    elif s.result == 'On Net':
        d['color'] = 'blue'
    elif s.result == 'Missed':
        d['color'] = 'orange'
    else:
        d['color'] = 'red'
    d['shape'] = 'circle'
    d['x'] = s.x_coord
    d['y'] = s.y_coord
    d['Details'] = s.result + s.shot_type + (", traffic" if s.traffic == 'True' else "") + (", one-timer" if s.one_timer == 'True' else "")
    event_list_dict.append(d)
    
    #shots against
    d = {}
    d['Event'] = 'Shots'
    d['Perspective'] = 'Against'
    if s.team_id.name == s.game_id.home_team.name:
        d['Team'] = s.game_id.away_team.name
    else:
        d['Team'] = s.game_id.home_team.name
    d['Game'] = str(s.game_id.game_id)
    d['Player'] = s.player_id.name
    if s.home_team_skaters == s.away_team_skaters:
        d['State'] = 'ES'
    elif (s.home_team_skaters > s.away_team_skaters):
        if (d['Team'] == s.game_id.home_team.name):
            d['State'] = 'PP'
        else:
            d['State'] = 'PK'
    else:
        if (d['Team'] == s.game_id.away_team.name):
            d['State'] = 'PK'
        else:
            d['State'] = 'PP'
    if s.result == 'Goal':
        d['color'] = 'green'
    elif s.result == 'On Net':
        d['color'] = 'blue'
    elif s.result == 'Missed':
        d['color'] = 'orange'
    else:
        d['color'] = 'red'
    d['shape'] = 'square'
    d['x'] = 200-s.x_coord
    d['y'] = 85-s.y_coord
    d['Details'] = s.result + s.shot_type + (", traffic" if s.traffic == 'True' else "") + (", one-timer" if s.one_timer == 'True' else "")       
    event_list_dict.append(d) 
    
#create dataframe and data source
df = pd.DataFrame(event_list_dict)
source = ColumnDataSource(df)

#create the tooltips
tooltips = [
    ('Event', '@Event'),
    ('Team', '@Team'),
    ('Player', '@Player'),
    ('State', '@State'),
    ('Perspective', '@Perspective'),
    ('Details', '@Details'),
]        

#inputs
events = Select(title="Event:", value="Shots", options=['Shots', 'Passes', 'Takeaways', 'Recoveries', 
                                                        'Penalties', 'Faceoffs', 'EE'])
teams = MultiSelect(title="Teams:", value=['Canada'], options=['Canada', 'Finland','ROC','United States'])
players = MultiChoice(title="Players:", value=['Marie-Philip Poulin'], options=['All'] + list(Players.objects.values_list("name", flat=True)))
vals = list(Games.objects.values_list("game_id", flat=True))
vals = [str(x) for x in vals]
games = MultiSelect(title="Games:", value=vals, options=vals)
state = Select(title="State:", value='All', options=['All','ES', 'PP', 'PK'])
perspective = Select(title="Perspective:", value='For', options=['For', 'Against'])

#graph
p = figure(height=510, title="", toolbar_location=None, tooltips=tooltips, width=1200)
url = "rink.png"
p.scatter(x='x', y='y', source=source, size=10, color='color', fill_alpha=0.5, marker='shape')
p.image_url(url=url, x=1, y=1)

#callbacks
# callback_events = CustomJS(args=dict(source=source), code="""
#                            var data = source.data;
                            
#                            """)

controls = [events, teams, players, games, state, perspective]
inputs = column(*controls, width=275, height=510)
layout = row(inputs, p)
output_file("test.html")
save(layout)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "url" value "rink.png" [renderer: GlyphRenderer(id='1657', ...)]


'/Users/hendrixhanes/Desktop/calgary_demo/database_files/test.html'